In [ ]:
# 引入 bokeh.plotting 作为 bpl，用于交互式数据可视化
import bokeh.plotting as bpl

# 引入 cv2，常用于计算机视觉任务
import cv2  

# 引入 glob，用于文件路径名的模式匹配
import glob  

# 从 IPython 导入 get_ipython，用于处理 IPython 内核的命令
from IPython import get_ipython 

# 引入 logging，用于记录日志信息
import logging 

# 引入 matplotlib.pyplot 作为 plt，用于绘制图形和进行数据可视化
import matplotlib.pyplot as plt

# 引入 numpy 作为 np，是 Python 的科学计算包
import numpy as np

# 引入 os，提供了与操作系统交互的功能
import os 

# 尝试设置 cv2 的线程数为 0，以优化性能
try:
    cv2.setNumThreads(0)  
except():
    pass

# 尝试在 IPython 环境中自动重新加载模块
try:
    if __IPYTHON__:        
        get_ipython().run_line_magic('load_ext', 'autoreload') 
        get_ipython().run_line_magic('autoreload', '2')
except NameError:  # 如果不在 IPython 环境中，则不执行任何操作
    pass

# 引入 caiman 作为 cm，这是一个用于钙信号数据分析的包
import caiman as cm  

# 从 caiman.motion_correction 导入 MotionCorrect，用于运动校正
from caiman.motion_correction import MotionCorrect      

# 从 caiman.source_extraction.cnmf 导入 cnmf 和 params，用于源提取和参数设置
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.source_extraction.cnmf import params as params

# 从 caiman.utils.utils 导入 download_demo，用于下载演示数据
from caiman.utils.utils import download_demo      

# 从 caiman.utils.visualization 导入绘制工具，用于数据的可视化
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour

# 设置 bokeh 输出到 Jupyter notebook
bpl.output_notebook()


In [ ]:
# 使用 logging 包进行日志管理
logging.basicConfig(format=
                          "%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s] [%(process)d] %(message)s",
                    # filename="/tmp/caiman.log", # 可以指定日志文件的路径和文件名，这里被注释掉了
                    level=logging.WARNING) # 设置日志级别为 WARNING


In [1]:
fnames = ['1_mcor.tif']  # 要處理的文件名

# 如果 fnames 列表中的第一個元素是 '1_mcor.tif' 或 'demoMovie.tif'，則執行以下操作
# 此行代碼用於檢查 '1_mcor.tif' 和 'demoMovie.tif' 是否為目標文件，若是，則將它們關聯起來
if fnames[0] in ['1_mcor.tif']: 
    fnames = [download_demo(fnames[0])]  # 使用 download_demo 函數下載對應的演示文件


NameError: name 'download_demo' is not defined

In [ ]:
display_movie = True  # 定義一個布林變量 display_movie，並設定其初始值為 True
if display_movie:  # 如果 display_movie 為 True，則執行下面的代碼塊
    m_orig = cm.load_movie_chain(fnames)  # 使用 caiman 包中的 load_movie_chain 函數載入電影鏈
    ds_ratio = 0.2  # 設定一個下採樣比率 ds_ratio 為 0.2
    m_orig.resize(1, 1, ds_ratio).play(  # 調整原始電影 m_orig 的大小，使用下採樣比率 ds_ratio，對載入的電影進行縮放並播放
        q_max=99.5, fr=30, magnification=2)  # 播放時設置最大品質 q_max 為 99.5，幀率 fr 為 30，放大倍數為 2



In [ ]:
# dataset dependent parameters
fr = 10                     # 影像速率，单位是每秒幀數
decay_time = 0.4            # 典型瞬态的持续时间，单位是秒

# motion correction parameters

strides = (48, 48)          # 每隔x像素开始一个新的pw-rigid运动校正补丁
overlaps = (24, 24)         # 补丁之间的重叠部分（补丁大小为strides+overlaps）
max_shifts = (6,6)          # 允许的最大刚性位移（以像素为单位）
max_deviation_rigid = 3     # 允许的针对刚性位移的最大补丁偏移
pw_rigid = True             # 执行非刚性运动校正的标志

# parameters for source extraction and deconvolution
p = 1                       # 自回归系统的阶数
gnb = 2                     # 全局背景成分的数量
merge_thr = 0.85            # 合并阈值，允许的最大相关性
rf = 15                     # 补丁的半尺寸，单位是像素。例如，如果rf=25，则补丁为50x50
stride_cnmf = 6             # 补丁之间的重叠量，单位是像素
K = 4                       # 每个补丁的组件数
gSig = [4, 4]               # 预期神经元的半尺寸，单位是像素
method_init = 'greedy_roi'  # 初始化方法（如果使用'sparse_nmf'分析树突数据）
ssub = 1                    # 初始化期间的空间子采样
tsub = 1                    # 初始化期间的时间子采样

# parameters for component evaluation
min_SNR = 2.0               # 接受一个组件的信噪比
rval_thr = 0.85             # 接受一个组件的空间相关性阈值
cnn_thr = 0.99              # 基于CNN的分类器的阈值
cnn_lowest = 0.1            # CNN概率低于此值的神经元将被拒绝


In [ ]:
# 定義一個參數字典，用於配置 CNMF 算法的各項參數
opts_dict = {
    'fnames': fnames,  # 文件名列表，用於指定要分析的影像文件
    'fr': fr,  # 影像的幀率（每秒幀數）
    'decay_time': decay_time,  # 信號衰減的時間
    'strides': strides,  # 在執行局部分解時的步長
    'overlaps': overlaps,  # 局部區域之間的重疊部分
    'max_shifts': max_shifts,  # 最大移動修正量（用於校正影像移動）
    'max_deviation_rigid': max_deviation_rigid,  # 剛性移動的最大偏差
    'pw_rigid': pw_rigid,  # 是否使用片段剛性移動修正
    'p': p,  # AR(p)模型中的參數p，用於時間序列的預測和去噪
    'nb': gnb,  # 背景成分的數量
    'rf': rf,  # 局部區域的半徑
    'K': K,  # 搜索的神經元數量
    'gSig': gSig,  # 空間過濾器的標準差
    'stride': stride_cnmf,  # 在執行 CNMF 時的步長
    'method_init': method_init,  # 初始化方法
    'rolling_sum': True,  # 是否使用滾動總和來估計背景
    'only_init': True,  # 是否只進行初始化，不執行後續分析
    'ssub': ssub,  # 空間上的降採樣因子
    'tsub': tsub,  # 時間上的降採樣因子
    'merge_thr': merge_thr,  # 合併閾值，用於合併相似的神經元成分
    'min_SNR': min_SNR,  # 最小信噪比，用於神經元分量的篩選
    'rval_thr': rval_thr,  # 相關性閾值，用於篩選神經元分量
    'use_cnn': True,  # 是否使用卷積神經網路進行分量篩選
    'min_cnn_thr': cnn_thr,  # CNN 篩選的最小閾值
    'cnn_lowest': cnn_lowest  # CNN 篩選的最低閾值
}

# 使用提供的參數字典創建 CNMF 參數對象
opts = params.CNMFParams(params_dict=opts_dict)


In [ ]:
# %% 開始一個用於並行處理的集群（如果已經存在一個集群，它將被關閉並開啟一個新的會話）
# 檢查 'dview' 變量是否在本地變量中。'dview' 通常用於代表當前的分佈式視圖
if 'dview' in locals():
    cm.stop_server(dview=dview)   # 如果存在，則停止服務器並關閉當前集群

# 設置新的集群。這裡使用 'cm.cluster.setup_cluster' 方法來配置和啟動一個新的集群
# backend='multiprocessing' 表示使用多進程後台進行並行計算
# n_processes=None 表示使用所有可用的處理器核心
# single_thread=False 表示每個進程可以使用多個線程
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='multiprocessing', n_processes=None, single_thread=False)


In [ ]:
# 首先我們根據指定的參數創建一個運動校正對象
mc = MotionCorrect(fnames, dview=dview, **opts.get_group('motion'))
# 注意，文件並未加載到內存中

# 解釋：
# `MotionCorrect` 是一個類別，用於運動校正。它通常用於處理視頻或圖像序列中的運動造成的扭曲。
# `fnames` 可能是一個文件名列表，代表要處理的視頻或圖像文件。
# `dview` 是一個分佈式處理視圖對象，用於並行處理。如果不提供，運動校正可能只在單個核心上運行。
# `opts.get_group('motion')` 可能是一個獲取運動校正相關參數的方法，`opts` 是一個配置對象，包含了不同處理步驟所需的參數。
# 註釋中提到的“文件並未加載到內存中”意味著運動校正過程是按需加載文件的，而不是一次性將所有文件加載到內存中，這有助於處理大文件或長視頻。


In [9]:
mc

In [ ]:
%%capture
# %%capture 可以捕捉并隐藏cell输出的魔法命令

#%% Run piecewise-rigid motion correction using NoRMCorre
# 使用 NoRMCorre 执行逐片刚性运动校正
mc.motion_correct(save_movie=True)
# mc.motion_correct 是一个运动校正方法，save_movie=True 表示保存校正后的影像

m_els = cm.load(mc.fname_tot_els)
# cm.load 加载处理后的影像数据，mc.fname_tot_els 是校正后影像的文件名

border_to_0 = 0 if mc.border_nan == 'copy' else mc.border_to_0 
# 设置边界值。如果 mc.border_nan 等于 'copy'，则边界设为0，否则使用 mc.border_to_0 的值
# 这里的 border_nan 和 border_to_0 是处理影像边界时使用的参数


In [ ]:
# %% compare with original movie 比較原始電影
display_movie = True  # 設定是否顯示電影
if display_movie:  # 如果需要顯示電影
    m_orig = cm.load_movie_chain(fnames)  # 加載原始電影數據，'cm' 可能是一个用於处理电影数据的包
    ds_ratio = 0.2  # 設定縮小比例，这里是将电影数据下采样（减少数据量）以便处理
    # 下面这行代码合并了调整大小后的原始电影和处理后的电影，然后进行播放
    cm.concatenate([m_orig.resize(1, 1, ds_ratio) - mc.min_mov*mc.nonneg_movie,
                    m_els.resize(1, 1, ds_ratio)], 
                   axis=2).play(fr=60, gain=15, magnification=2, offset=0)  # 按 q 退出播放
    # 'cm.concatenate' 是合併兩個電影數據的函數
    # 'resize' 函数改变电影数据的大小
    # 'play' 函数用于播放合并后的电影，'fr' 是帧率, 'gain' 是增益, 'magnification' 是放大倍数, 'offset' 是偏移量


In [ ]:
#%% MEMORY MAPPING
# memory map the file in order 'C'
fname_new = cm.save_memmap(mc.mmap_file, base_name='NT5-1', # name change
                           order='C', border_to_0=border_to_0, dview=dview) 
    # 將檔案以 'C' 順序映射到記憶體中
    # cm.save_memmap 是用來儲存記憶體映射檔案的函數，mc.mmap_file 是要映射的檔案
    # base_name='NT5-1' 表示設定映射檔案的基本名稱，order='C' 表示以C語言風格的行主序來儲存陣列
    # border_to_0 和 dview 是額外的參數，用於控制映射過程

# now load the file
Yr, dims, T = cm.load_memmap(fname_new)
    # 加載剛才創建的記憶體映射檔案
    # cm.load_memmap 是用來加載記憶體映射檔案的函數
    # Yr 是映射檔案的數據陣列，dims 是數據的維度，T 是時間軸的長度

images = np.reshape(Yr.T, [T] + list(dims), order='F') 
    # 將加載的數據重新排列成圖像格式 (時間軸 x 寬度 x 高度)
    # np.reshape 是 NumPy 函數，用於重新排列數據的形狀
    # Yr.T 是對 Yr 進行轉置，使得時間軸 T 成為第一維
    # [T] + list(dims) 設定了新的數據形狀，order='F' 表示以Fortran風格的列主序來儲存陣列


In [ ]:
# %% 重啟集群以清理記憶體
cm.stop_server(dview=dview)  # 呼叫 cm (caiman) 套件停止伺服器，釋放由 'dview' 指定的資源

# 設定集群
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='multiprocessing',  # 使用 'multiprocessing' 作為後端
    n_processes=None,  # 自動決定進程數量
    single_thread=False)  # 設定為多線程模式


In [ ]:
%%capture
# 使用 %%capture 魔術指令來捕捉該單元格的輸出，防止輸出太多信息到屏幕上

#%% RUN CNMF ON PATCHES
# 在小片段上運行 CNMF（Constrained Nonnegative Matrix Factorization，受限非負矩陣分解）
# 首先在小片段上提取空間和時間組件，然後將它們結合起來
# 在這個步驟中，去捲積被關閉（p=0）。如果你想在每個小片段內進行去捲積，
# 可以將 params.patch['p_patch'] 的值改為非零

cnm = cnmf.CNMF(n_processes, params=opts, dview=dview)  # 創建 CNMF 對象
cnm = cnm.fit(images)  # 對圖像應用 CNMF，提取空間和時間組件


In [ ]:
# %% plot contours of found components
# 繪製發現的組件的輪廓

# cm 是用於分析和處理視頻數據的一個庫，這裡的 local_correlations 函數計算影像的局部相關性
Cn = cm.local_correlations(images.transpose(1,2,0))

# 處理任何NaN值（不是數值），將它們設置為0，以便進行穩定的影像處理
Cn[np.isnan(Cn)] = 0

# cnm 是一個用於鈣信號分析的對象。這裡的 estimates.plot_contours_nb 函數繪製影像Cn上發現的組件的輪廓
cnm.estimates.plot_contours_nb(img=Cn)


In [ ]:
%%capture
# 使用 %%capture 魔法命令來捕獲此單元格的輸出，以防止其顯示在Jupyter Notebook的輸出中。

#%% RE-RUN seeded CNMF （限制非负矩阵分解） on accepted patches to refine and perform deconvolution 
# 重新运行带有种子的CNMF（Constrained Non-negative Matrix Factorization，限制非负矩阵分解）在接受的补丁上以精炼和执行去卷积。
# 这是用于图像或视频数据的一种分析方法，特别适用于神经成像数据。

cnm2 = cnm.refit(images, dview=dview)
# 使用 refit 方法对图像进行再拟合，以优化之前的CNMF模型。
# 'images' 是要处理的图像数据。
# 'dview' 是一个用于并行计算的视图对象，可能是与并行处理库（如ipyparallel）相关的。


In [ ]:
# 这段代码中，使用了 CNMF（限制非负矩阵分解）的方法。这是一种分析技术，通常用于神经科学领域，
# 尤其是在处理类似于脑部成像数据的复杂数据集时。它能够从这些数据中提取有用的信号，这在研究神
# 经元活动时特别重要。代码中的 refit 方法用于优化和改进之前的CNMF分析结果，这在处理动态或时
# 变数据时非常有用。同时，使用并行视图对象 dview 可以加速这个过程，特别是在处理大量数据时。

In [ ]:
#%% COMPONENT EVALUATION
# 這部分代碼是用於評估組件的
# 組件的評估有三種方式：
#   a) 每個組件的形狀必須與數據相關聯
#   b) 在瞬變事件的長度上需要一個最小的峰值信噪比
#   c) 每個形狀都要通過基於CNN的分類器

cnm2.estimates.evaluate_components(images, cnm2.params, dview=dview)


In [ ]:
#%% PLOT COMPONENTS
# 使用 cnm2.estimates.plot_contours_nb 函数来绘制轮廓图。
# img 参数设置为 Cn，这通常是一个背景图像或相关参考图像。
# idx 参数设置为 cnm2.estimates.idx_components，表示我们要在图像上标出的特定组件的索引。
cnm2.estimates.plot_contours_nb(img=Cn, idx=cnm2.estimates.idx_components);


In [ ]:
# accepted components
# cnm2.estimates.nb_view_components() 函式用於显示组件的图像
# 'img=Cn' 参数代表要显示的图像
# 'idx=cnm2.estimates.idx_components' 参数指定要显示的组件的索引
cnm2.estimates.nb_view_components(img=Cn, idx=cnm2.estimates.idx_components);


In [ ]:
# rejected components
# 檢查被拒絕的組件
if len(cnm2.estimates.idx_components_bad) > 0:
    # 若存在被拒絕的組件，則顯示它們
    cnm2.estimates.nb_view_components(img=Cn, idx=cnm2.estimates.idx_components_bad)
else:
    # 若無被拒絕的組件，則輸出相應信息
    print("No components were rejected.")


In [ ]:
#%% Extract DF/F values
# 提取 ΔF/F 值
# ΔF/F 是一个常用于神经科学中的计算方法，用于表示钙信号的相对变化。
# 这个步骤通常在钙影像数据分析中非常重要，用于量化神经元活动。

cnm2.estimates.detrend_df_f(quantileMin=8, frames_window=250);
# cnm2 是一个对象，通常与钙影像数据分析库（如CaImAn）相关。
# .estimates 是CaImAn中用于存储和处理估计结果的属性。
# detrend_df_f() 是一个方法，用于去除趋势并计算 ΔF/F。
# quantileMin 参数设置去趋势时使用的最小分位数。
# frames_window 参数定义了用于计算基线的帧窗口大小。


In [ ]:
# 使用 cnm2 的 estimates 属性中的 select_components 方法
# 选择特定的组件。此处设置 use_object=True 表明使用面向对象的方式进行选择。
cnm2.estimates.select_components(use_object=True);


In [ ]:
# 导入所需的包
# cnm2 专门的图像处理或神经网络模型库的实例或对象

# 使用 cnm2 的 estimates 属性中的 nb_view_components 方法
# 这个方法用于查看图像组件
# 参数 img=Cn 指定要处理的图像
# 参数 denoised_color='red' 指定去噪声后的颜色显示为红色
cnm2.estimates.nb_view_components(img=Cn, denoised_color='red')

# 打印提示信息
# 提示可能需要改变数据传输率以生成图像
# 使用 Jupyter Notebook 时，通过设置 --NotebookApp.iopub_data_rate_limit=1.0e 来调整
print('you may need to change the data rate to generate this one: use jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e')


In [ ]:
# 將 save_results 設定為 True，以決定是否儲存結果
save_results = True

# 如果 save_results 為 True，則執行儲存操作
if save_results:
    # 使用 cnm2.save 函數將分析結果儲存為 HDF5 格式的文件
    # HDF5 格式是一種用於儲存大量科學數據的文件格式
    cnm2.save('analysis_results.hdf5')   # 存储路径：C:\Users\HSH Lab\analysis_results.hdf5


In [ ]:
# %% 停止集群并清理日志文件
# 使用 `cm.stop_server` 方法停止服务器。这里的 `dview` 变量可能是与集群服务器相关的一个参数。
cm.stop_server(dview=dview)

# 寻找当前目录下所有符合 '*_LOG_*' 模式的文件。这通常用于匹配特定的日志文件。
log_files = glob.glob('*_LOG_*')

# 遍历找到的日志文件列表
for log_file in log_files:
    # 删除每一个日志文件。使用 `os.remove` 函数来删除文件。
    os.remove(log_file)


In [ ]:
cnm2.estimates.play_movie(images, q_max=99.9, gain_res=2,
                                  magnification=2,
                                  bpx=border_to_0,
                                  include_bck=False);

# 使用cnm2模块的estimates.play_movie函数进行电影处理
# images - 传入的电影帧数据
# q_max - 可选参数，表示质量因子的最大值，默认为99.9
# gain_res - 可选参数，表示增益因子的分辨率，默认为2
# magnification - 可选参数，表示放大倍数，默认为2
# bpx - 可选参数，表示边界像素，默认为border_to_0
# include_bck - 可选参数，是否包括背景处理，默认为False


In [ ]:
#%% reconstruct denoised movie 使用cm.movie函数对信号进行去噪
denoised = cm.movie(cnm2.estimates.A.dot(cnm2.estimates.C) + \
                    cnm2.estimates.b.dot(cnm2.estimates.f)).reshape(dims + (-1,), order='F').transpose([2, 0, 1])

In [ ]:
# 代码中使用的包/模块的功能解释：
# - cm：自定义的包，用于处理电影数据，可能包含了去噪功能。
# - cnm2：这个包或模块可能包含了一些关于非负矩阵分解（NMF）的估计，如A、C、b和f等参数的估计。
# - dot：这是一个矩阵相乘的函数，用于计算A.dot(C)和b.dot(f)的结果。
# - reshape：这个函数用于改变数组的形状，将计算结果调整成dims的形状。
# - order='F'：这是reshape函数的参数，表示使用Fortran风格的顺序重新排列数组元素。
# - transpose：这个函数用于交换数组的轴，将结果调整成所需的形状，[2, 0, 1]表示交换轴的顺序。

# 请注意，具体的功能和用法可能需要根据上下文和实际代码的用途来确认。


In [ ]:
denoised

In [ ]:
from PIL import Image
import numpy as np
import caiman as cm

# 确保cnm2是CNMF算法运行后的结果对象, cnm2.estimates.A和cnm2.estimates.C计算好

# 获取重建的电影数据
reconstructed_movie = cm.movie(cnm2.estimates.A.dot(cnm2.estimates.C))

# 将重建的电影转换为NumPy数组
reconstructed_movie_array = np.array(reconstructed_movie)

images = []

# 遍历每一帧，将其转换为PIL图像，并添加到列表中
for i in range(reconstructed_movie_array.shape[0]):
    frame = reconstructed_movie_array[i]
    images.append(Image.fromarray(frame.astype(np.uint8)))

# 确保至少有一帧图像
if images:
    # 保存第一帧，并将其余帧作为TIF文件的其他页
    tif_path = 'reconstructed_movie.tif'
    images[0].save(tif_path, save_all=True, append_images=images[1:], compression="tiff_deflate")

print(f"The reconstructed movie has been saved to {tif_path}")
